### RNN的反向传播算法 - Backpropagation for RNNs


RNN 中的 BP 算法本质上和全连接中的BP算法是一样的，只不过因为 RNN 和时间有关，而且每个时间点的权重是共享的，导致RNN中的BP算法稍显复杂，叫做 <b>backpropagation through time, <span class="motutor-highlight motutor-id_rfvjrxu-id_m5p445e"><i></i>BPTT</span></b>。

RNN 从一个时间步传播权值矩阵到下一个时间步。例如，考虑以下两个句子：

![](https://imgbed.momodel.cn/%E5%8F%8D%E5%90%91%E4%BE%8B%E5%AD%90.png)


对上面的两个句子，根据上下文，都可以知道空白处的答案是 “John”,第二个在两个句子的上下文中均提及了好几次的人。在实际中，RNN 预测句子中的空白处答案正确可能性，第一句要比第二句高。  
这是因为在反向传播的阶段的过程中，从前面时间步中回传过来的梯度值会逐渐消失。因此，对于长句子，预测到 “John” 是空白处的答案的概率会随着上下文信息增大而减少。

考虑公式在时间步 t ，计算 RNN 误差 $\frac{dE}{dW}$ ，然后我们把每个时间步的误差都加起来。也就是说，计算并累积每个时间步长 t 的 $\frac{dE_t}{dW}$


$$
\frac{\partial E}{\partial W}=\sum_{i=1}^{T} \frac{\partial E_{t}}{\partial W}
\tag{10}
$$

通过将微分链式法来计算每个时间步长的误差；公式 (11) 展示对应的微分计算。注意 $\frac{d h_{t}}{d h_{k}}$ 是 $h_t$ 对之前所有的 k 个时间步的偏导数。

$$
\frac{\partial E_{t}}{\partial W}=\sum_{k=1}^{T} \frac{\partial E_{t}}{\partial y_{t}} \frac{\partial y_{t}}{\partial h_{t}} \frac{\partial h_{t}}{\partial h_{k}} \frac{\partial h_{k}}{\partial W}
\tag{11}
$$


下式 展示了计算每个 $\frac{d h_{t}}{d h_{k}}$ 的关系；这是在时间间隔 $[k,t]$ 内对所有的隐藏层的应用一个简单的微分链式法则。

$$
\frac{\partial h_{t}}{\partial h_{k}}=\prod_{j=k+1}^{t} \frac{\partial h_{j}}{\partial h_{j-1}}=\prod_{j=k+1}^{t} W^{T} \times \operatorname{diag}[f'\left(j_{j-1}\right)]
\tag{12}
$$

因为 $h \in \mathbb{R}^{D\_{n}}$ ，每个 $\frac{\partial h\_{j}}{\partial h\_{j-1}}$ 是 h 的 Jacobian 矩阵的元素：

$$
\frac{\partial h_{j}}{\partial h_{j-1}}=\left[\frac{\partial h_{j}}{\partial h_{j-1,1}} \cdots \frac{\partial h_{j}}{\partial h_{j-1, D_{n}}}\right]=\left[\begin{array}{ccc}{\frac{\partial h_{j, 1}}{\partial h_{j-1,1}}} & {\cdots} & {\frac{\partial h_{j,1}}{\partial h_{j-1, D\_{n}}}} \\ {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial h_{j, D_{n}}}{\partial h_{j - 1,1}}} & {\cdots} & {\frac{\partial h_{j, D_{n}}}{\partial h_{j-1, D_{n}}}}\end{array}\right]
\tag{13}
$$

将公式合起来，我们有以下关系。

$$
\frac{\partial E}{\partial W}=\sum_{t=1}^{T} \sum_{k=1}^{t} \frac{\partial E_{t}}{\partial y_{t}} \frac{\partial y_{t}}{\partial h_{t}}\left(\prod_{j=k+1}^{t} \frac{\partial h_{j}}{\partial h_{j-1}}\right) \frac{\partial h_{k}}{\partial W}
\tag{14}
$$

下式展示了 Jacobian 矩阵的范数。这里的 $\beta_{W}$ 和 $\beta_{h}$ 是这两个矩阵范数的上界值。因此通过公式所示的关系计算在每个时间步 t 的部分梯度范数。

$$
\left\|\frac{\partial h\_{j}}{\partial h_{j-1}}\right\| \leq\left\|W^{T}\right\|\left\|\operatorname{diag}\left[f^{\prime}\left(h_{j-1}\right)\right]\right\| \leq \beta_{W} \beta_{h} 
\tag{15}
$$

计算这两个矩阵的 L2 范数。在给定的非线性函数 sigmoid 下， $f^{\prime}\left(h_{j-1}\right)$ 的范数只能等于 1。

$$
\left\|\frac{\partial h_{t}}{\partial h_{k}}\right\|=\left\|\prod_{j=k+1}^{t} \frac{\partial h_{j}}{\partial h_{j-1}}\right\| \leq\left(\beta_{W} \beta\_{h}\right)^{t-k}
\tag{16}
$$


当 $t - k$ 足够大和 $\beta_{W} \beta_{h}$ 远远小于 1 或者远远大于 1，指数项 $\left(\beta_{W} \beta_{h}\right)^{t-k}$ 的值就很容易变得非常小或者非常大。由于单词之间的距离过大，用一个很大的 $t-k$ 评估交叉熵误差可能会出现问题。在反向传播的早期就出现梯度消失，那么远处单词对在时间步长 $t$ 预测下一个单词中，所起到的作用就会变得很小。


在实验的过程中，一旦梯度的值变得非常大，会导致在运行过程中容易检测到其引起的溢出（即 NaN）；这样的问题称为梯度爆炸问题。然而，当梯度接近为 0 的时候，梯度近乎不再存在，同时降低模型对语料库中的远距离的单词的学习质量；这样的问题称为梯度消失问题。


如果想对梯度消失问题的有更直观的了解，你可以访问这个[样例网站](https://link.zhihu.com/?target=http%3A//cs224d.stanford.edu/notebooks/vanishing_grad_example.html)。

对RNN梯度消失和爆炸的更多解释可以参考 https://zhuanlan.zhihu.com/p/28687529
